In [1]:
func_dir = '/global/home/users/ann_scheliga/CYGNSS-GRACE/codebase/'
data_dir = '/global/scratch/users/ann_scheliga/dam_datasets/'

In [2]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
list_of_filenames = os.listdir(data_dir)
LH_usbr_filename = [fname for fname in list_of_filenames if 'havasu' in fname.lower() and 'usbr' in fname.lower()][0]
LH_cdec_filename = [fname for fname in list_of_filenames if 'havasu' in fname.lower() and 'cdec' in fname.lower()][0]
LM_usbr_filename = [fname for fname in list_of_filenames if 'mohave' in fname.lower() and 'usbr' in fname.lower()][0]
LM_cdec_filename = [fname for fname in list_of_filenames if 'mohave' in fname.lower() and 'cdec' in fname.lower()][0]

In [4]:
LH_usbr_raw = pd.read_csv(data_dir + LH_usbr_filename,header=7)
LM_usbr_raw = pd.read_csv(data_dir + LM_usbr_filename,header=7)
LH_cdec_raw = pd.read_csv(data_dir + LH_cdec_filename)
LM_cdec_raw = pd.read_csv(data_dir + LM_cdec_filename)

In [5]:
def convert_to_num(single_value):
    if isinstance(single_value , (float,int)):
        return single_value
    try:
        float(single_value.replace(',',''))
    except:
        return np.nan
    else:
        return float(single_value.replace(',',''))

In [6]:
def cdec_dataprocessing(df):
    df['DATE'] = pd.to_datetime(df['DATE'])
    df['STORAGE AF'] = df['STORAGE AF'].apply(convert_to_num)
    return df

In [7]:
def usbr_dataprocessing(df):
    df = df.drop(columns='Location')
    df.dropna(axis=0,how='any',inplace=True)
    df =  df[df['Timestep']!='Timestep'] # remove repeat header rows
    df['Variable'] =  df['Parameter'] + ' [' + df['Units'] + ']'
    df['Result'] =    df['Result'].apply(convert_to_num)
    df_pivot = df.pivot(columns='Variable',index='Datetime (UTC)',values='Result')
    return df_pivot

In [32]:
def load_usbr_data(name,file_dir='/global/scratch/users/ann_scheliga/dam_datasets/'):
    try:
        # test if `name` input is a filename
        pd.read_csv(file_dir+name)
    except Exception:
        # if `name` input is a reservoir name, find the filename in the folder
        # list comp looks for reservoir name and "usbr" in file strings
        list_of_filenames = os.listdir(file_dir)
        filename = [fname for fname in list_of_filenames if name.lower() in fname.lower() and 'usbr' in fname.lower()][0]
    else:
        filename = name
    finally:
        raw_data = pd.read_csv(file_dir + filename, header = 7)
    return raw_data

In [33]:
def load_formatted_usbr_data(name,file_dir='/global/scratch/users/ann_scheliga/dam_datasets/'):
    raw_data = load_usbr_data(name,file_dir)
    data = usbr_dataprocessing(raw_data)
    return data

In [34]:
load_formatted_usbr_data('MOhave')

Variable,Lake/Reservoir Elevation [ft],Lake/Reservoir Release - Total [af],Lake/Reservoir Release - Total [cfs],Lake/Reservoir Storage [af],Water Temperature [DegF]
Datetime (UTC),,,,,
2018-01-01 07:00:00,641.045,12891.90,6499.67,1645620.0,70.9285
2018-01-02 07:00:00,641.403,9118.10,4597.04,1655280.0,70.7430
2018-01-03 07:00:00,641.730,10890.00,5490.38,1664110.0,70.5513
2018-01-04 07:00:00,642.106,9099.83,4587.83,1674290.0,70.3678
2018-01-05 07:00:00,642.374,9105.95,4590.92,1681610.0,70.1882
...,...,...,...,...,...
2022-12-28 07:00:00,639.891,19089.80,9624.42,1614590.0,71.0042
2022-12-29 07:00:00,639.905,14285.10,7202.08,1614960.0,70.8353
2022-12-30 07:00:00,639.888,11901.90,6000.54,1614510.0,70.6636


In [8]:
LH_cdec = cdec_dataprocessing(LH_cdec_raw)
LM_cdec = cdec_dataprocessing(LM_cdec_raw)

LH_usbr = usbr_dataprocessing(LH_usbr_raw)
LM_usbr = usbr_dataprocessing(LM_usbr_raw)

In [9]:
LH_usbr

Variable,Lake/Reservoir Elevation [ft],Lake/Reservoir Release - Total [af],Lake/Reservoir Release - Total [cfs],Lake/Reservoir Storage [af],Water Temperature [DegF]
Datetime (UTC),,,,,
2018-01-01 07:00:00,446.888,9734.71,4907.92,559039.0,52.5975
2018-01-02 07:00:00,447.018,5774.38,2911.25,561440.0,52.6581
2018-01-03 07:00:00,447.015,5576.94,2811.71,561384.0,52.6735
2018-01-04 07:00:00,447.152,3569.92,1799.83,563973.0,52.6800
2018-01-05 07:00:00,446.985,7915.95,3990.96,560824.0,52.8633
...,...,...,...,...,...
2022-12-28 07:00:00,446.321,11898.70,5998.92,548606.0,54.4556
2022-12-29 07:00:00,446.842,3594.30,1812.12,558193.0,54.5398
2022-12-30 07:00:00,447.024,5153.80,2598.38,561554.0,54.4244
